In [ ]:
!pip install -q gradio

In [ ]:
!pip install soundfile

In [ ]:
pip install TTS API

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.0/938.0 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install transformers
!pip install --upgrade numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 65.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.0
    Uninstalling numpy-1.22.0:
      Successfully uninstalled numpy-1.22.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.22.0 requires numpy==1.22.0; python_version <= "3.10", but you have numpy 1.26.4 which is incompatible.


In [ ]:
import os
import csv
import librosa
import torch
from transformers import pipeline
from google.colab import drive

# Initialize the transcription pipeline
model_name = "openai/whisper-large-v3"
device = 0 if torch.cuda.is_available() else -1
pipe = pipeline("automatic-speech-recognition", model=model_name, device=device)

def translate_audio(audio_path, target_sr=16000, max_duration=None, task='translate',language="en"):
    # Load the audio file, ensuring it's at the target sample rate
    audio, sr = librosa.load(audio_path, sr=target_sr)
    # Trim the audio to max_duration if specified
    if max_duration:
        max_samples = max_duration * target_sr
        if len(audio) > max_samples:
            audio = audio[:max_samples]
    # Transcribe the audio, getting both text and timestamps
    translation_result = pipe(audio, generate_kwargs={"task": task, "language":language})
    translation_text = translation_result["text"]
    #transcription_result_wl = pipe(audio, generate_kwargs={"task": task, "language": "ar"}, return_timestamps="word")

    # Assuming word-level timestamps can be obtained or approximated
    #word_timestamps = transcription_result_wl.get("chunks", [])
    return translation_text

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 Thank you.


In [13]:
import gradio as gr
from TTS.api import TTS

# getting the latest XTTS_v2
tts = TTS("xtts", gpu=True)
def gradio_process(uploaded_audio_path, source_language, target_language, action, user_translation=""):
    global latest_translation

    if action == "Translate":
        # Perform translation
        try:
            translation = translate_audio(audio_path=uploaded_audio_path, language=source_language)
            latest_translation = translation  # Save the translation for potential cloning
            return translation, None
        except Exception as e:
            latest_translation = ""  # Clear the latest translation on error
            return f"An error occurred during translation: {str(e)}", None
    elif action == "Clone":
        # Perform voice cloning using the provided user translation or the latest translation
        text_to_clone = user_translation if user_translation else latest_translation
        if not text_to_clone:
            return "No text to clone. Please translate first.", None

        # Perform voice cloning
        try:
            output_audio_path = "/tmp/cloned_voice_output.wav"
            tts.tts_to_file(text=text_to_clone,
                            file_path=output_audio_path,
                            speaker_wav=uploaded_audio_path,
                            language=target_language)
            return text_to_clone, output_audio_path
        except Exception as e:
            return f"An error occurred during voice cloning: {str(e)}", None

# Set up the Gradio interface
iface = gr.Interface(
    fn=gradio_process,
    inputs=[
        gr.Audio(label="Upload your audio file (WAV format)", type="filepath"),
        gr.Dropdown(choices=["en", "ar"], label="Source Language"),
        gr.Dropdown(choices=["en", "ar"], label="Target Language"),
        gr.Radio(choices=["Translate", "Clone"], label="Action"),
        gr.Textbox(label="Enter translation here if you wish to modify it before cloning"),
    ],
    outputs=[
        gr.Textbox(label="Translated Text"),
        gr.Audio(label="Cloned Voice Audio"),
    ],
    title="DupAI: Audio Translation and Voice Cloning",
    description="Upload an audio file and select the source and target language to translate. You can then clone the voice in the translated language."
)

iface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://9cccf2b28d5cfdc5bd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9cccf2b28d5cfdc5bd.gradio.live
